In [1]:
import os
import torch
import numpy as np
from collections import defaultdict
from utils.load_results import *
from utils.analysis_from_interaction import *

In [2]:
datasets = ('(3,4)', '(3,8)', '(3,16)', '(4,4)', '(4,8)', '(5,4)')
n_attributes = (3, 3, 3, 4, 4, 5)
n_values = (4, 8, 16, 4, 8, 4)
n_epochs = 300
paths = [f'results/vague_ds_results/{d}_game_size_10_vsf_3/' for d in datasets]

In [3]:
context_unaware = False # whether original or context_unaware simulations are evaluated
if context_unaware:
    setting = 'context_unaware'
else:
    setting = 'standard'

# Determine Unique Message size 

In [4]:
from collections import Counter

##### Total message size if symbol order matters:

In [9]:
num_runs = 5 # number of runs per ds
for i, d in enumerate(datasets):
    print(f"Dataset: {d}")
    unique_messages_all_runs = []

    for run in range(num_runs):
        path_to_run = paths[i] + '/' + str(setting) + '/' + str(run) + '/'
        path_to_interaction_train = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
        interaction = torch.load(path_to_interaction_train)

        messages = interaction.message.argmax(dim=-1)
        messages = [tuple(msg.tolist()) for msg in messages]  # Convert messages to tuples for hashing
        total_messages = set(messages)  # Set removes duplicated concepts. Note: tuples are returned unordered  
        number_of_unique_messages = len(total_messages)
        unique_messages_all_runs.append(number_of_unique_messages)

    # Calculate summary statistics for unique messages
    mean_unique_messages = np.mean(unique_messages_all_runs)

    print(f'Number of unique messages per run: {unique_messages_all_runs}')
    print(f'Mean unique messages: {mean_unique_messages:.2f}')
    print()
          

Dataset: (3,4)
Number of unique messages per run: [119, 90, 73, 78, 152]
Mean unique messages: 102.40

Dataset: (3,8)
Number of unique messages per run: [439, 260, 694, 507, 835]
Mean unique messages: 547.00

Dataset: (3,16)
Number of unique messages per run: [1483, 2147, 1640, 2876, 2765]
Mean unique messages: 2182.20

Dataset: (4,4)
Number of unique messages per run: [718, 1762, 180, 765, 906]
Mean unique messages: 866.20

Dataset: (4,8)
Number of unique messages per run: [16501, 5310, 4778, 8900, 7268]
Mean unique messages: 8551.40

Dataset: (5,4)
Number of unique messages per run: [7437, 5055, 5651, 11539, 6889]
Mean unique messages: 7314.20



##### Unique message size if you consider messages with the same symbols but in different orders as the same message:

In [12]:
unique_sorted_messages_all_datasets = []

for i, d in enumerate(datasets):
    print(f"Dataset: {d}")
    unique_sorted_messages_all_runs = []

    for run in range(num_runs):
        path_to_run = paths[i] + '/' + str(setting) + '/' + str(run) + '/'
        path_to_interaction_train = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
        interaction = torch.load(path_to_interaction_train)

        messages = interaction.message.argmax(dim=-1)
         # Convert messages to sorted tuples. Sorted orders ints in ascending order.
        sorted_messages = [tuple(sorted(msg.tolist())) for msg in messages] 
        # Set removed duplicates. 
        unique_sorted_messages = set(sorted_messages)
        number_of_unique_sorted_messages = len(unique_sorted_messages) 
        unique_sorted_messages_all_runs.append(number_of_unique_sorted_messages)
    
    unique_sorted_messages_all_datasets.append(unique_sorted_messages_all_runs)
    mean_unique_sorted_messages = np.mean(unique_sorted_messages_all_runs)
    print(f'Number of unique messages per run: {unique_sorted_messages_all_runs}')
    print(f'Mean unique sorted messages: {mean_unique_sorted_messages:.2f}')
    print() 

Dataset: (3,4)
Number of unique messages per run: [90, 60, 59, 58, 95]
Mean unique sorted messages: 72.40

Dataset: (3,8)
Number of unique messages per run: [297, 225, 495, 390, 609]
Mean unique sorted messages: 403.20

Dataset: (3,16)
Number of unique messages per run: [1358, 1705, 1416, 2569, 2343]
Mean unique sorted messages: 1878.20

Dataset: (4,4)
Number of unique messages per run: [443, 1098, 145, 498, 490]
Mean unique sorted messages: 534.80

Dataset: (4,8)
Number of unique messages per run: [9395, 4297, 3421, 6000, 4655]
Mean unique sorted messages: 5553.60

Dataset: (5,4)
Number of unique messages per run: [3599, 2793, 3434, 4914, 3066]
Mean unique sorted messages: 3561.20



##### Calculates unique message size if you consider messages with the same symbols but in different orders as the same message. Then draws comparison between unique sorted messages with the number of unique concepts:

In [15]:
unique_sorted_messages_all_datasets = []
unique_concepts_all_datasets = []

for i, d in enumerate(datasets):
    print(f"Dataset: {d}")
    unique_sorted_messages_all_runs = []
    unique_concepts_all_runs = []
    
    for run in range(num_runs):
        path_to_run = paths[i] + '/' + str(setting) + '/' + str(run) + '/'
        path_to_interaction_train = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
        interaction = torch.load(path_to_interaction_train)
        
        # Calculate unique sorted messages
        messages = interaction.message.argmax(dim=-1)
        sorted_messages = [tuple(sorted(msg.tolist())) for msg in messages]
        unique_sorted_messages = set(sorted_messages)
        number_of_unique_sorted_messages = len(unique_sorted_messages)
        unique_sorted_messages_all_runs.append(number_of_unique_sorted_messages)
        
        # Calculate unique concepts
        sender_input = interaction.sender_input
        n_targets = int(sender_input.shape[1]/2)
        target_objects = sender_input[:, :n_targets]
        target_objects = k_hot_to_attributes(target_objects, n_values[i])
        (objects, fixed) = retrieve_concepts_sampling(target_objects, all_targets=True)
        concepts = list(zip(objects, fixed))
        concepts_strings = [(str(obj), str(fixed_vec)) for obj, fixed_vec in concepts]
        unique_concepts = set(concepts_strings)
        number_of_unique_concepts = len(unique_concepts)
        unique_concepts_all_runs.append(number_of_unique_concepts)
        
    unique_sorted_messages_all_datasets.append(unique_sorted_messages_all_runs)
    unique_concepts_all_datasets.append(unique_concepts_all_runs)
    
    print(f'Number of unique messages per run: {unique_sorted_messages_all_runs}')
    print(f'Number of unique concepts per run: {unique_concepts_all_runs}')
    print()

# Calculate the ratio for each run in each dataset
for i, d in enumerate(datasets):
    print(f"Dataset: {d}")
    ratios = []
    
    for run in range(num_runs):
        num_unique_messages = unique_sorted_messages_all_datasets[i][run]
        num_unique_concepts = unique_concepts_all_datasets[i][run]
        ratio = num_unique_messages / num_unique_concepts
        ratios.append(ratio)
        
    print(f'Ratios of unique messages to unique concepts per run: {ratios}')
    mean_ratio = np.mean(ratios)
    print(f'Mean ratio: {mean_ratio:.2f}')
    print()

Dataset: (3,4)
Number of unique messages per run: [90, 60, 59, 58, 95]
Number of unique concepts per run: [715, 688, 683, 688, 672]

Dataset: (3,8)
Number of unique messages per run: [297, 225, 495, 390, 609]
Number of unique concepts per run: [2850, 2971, 2848, 2819, 2854]

Dataset: (3,16)
Number of unique messages per run: [1358, 1705, 1416, 2569, 2343]
Number of unique concepts per run: [12750, 12564, 12797, 12971, 12681]

Dataset: (4,4)
Number of unique messages per run: [443, 1098, 145, 498, 490]
Number of unique concepts per run: [5830, 5995, 5935, 6153, 6137]

Dataset: (4,8)
Number of unique messages per run: [9395, 4297, 3421, 6000, 4655]
Number of unique concepts per run: [45246, 45026, 45086, 45252, 44819]

Dataset: (5,4)
Number of unique messages per run: [3599, 2793, 3434, 4914, 3066]
Number of unique concepts per run: [45433, 45306, 45401, 45032, 45226]

Dataset: (3,4)
Ratios of unique messages to unique concepts per run: [0.1258741258741259, 0.0872093023255814, 0.08638360